In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


In [2]:
%pwd
%cd ..
%cd src
%pwd

/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM
/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src


/home/nas4_user/hojuncho/.cache/pypoetry/virtualenvs/anomllm-yItPHLz9-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src'

In [3]:
#Please edit the BASE_DIR on your situation.

BASE_DIR = "/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM"

In [4]:
def make_localization_prompt(anomal_data):
        
    LOCALIZATION_PROMPT = f"""Detect the ranges of anomalies in this time series based on the x-axis coordinate, where the x-axis coordinate refers to the x value in (x, y) data. The patterns of anomalies are as follows.
    {anomal_data}\nList one by one, in JSON format. If there are no anomalies, answer with an empty list [].

    Output template:
    [{{"start": ..., "end": ...}}, {{"start": ..., "end": ...}}...]
    """
    return LOCALIZATION_PROMPT

In [5]:
def create_gt_data(anomaly_locations)->list :
    gt_data = []
    for i in anomaly_locations:
        anomal = i[0]
        gt_data.append([[int(x) for x in sublist] for sublist in anomal])
    return gt_data

In [6]:
#raw_text = raw_results['gpt-4o-mini (0shot-text)']['request']['messages'][0]['content'][0]['text']
#data_part = raw_text.split("\n\n")[0]
#data_part_list = data_part.
import numpy as np
from prompt import time_series_to_str

def create_idx_datalist(arr:np.ndarray)->list:
    """
    입력: array([[ 0.00000000e+00],
       [ 1.87381312e-01],
       [ 3.68124545e-01]]) 형태.
    출력: [(0, '0.0'),
            (1, '0.19'),
            (2, '0.37')] 형태.
    """
    idx_data_list  = []
    for i, data in enumerate(time_series_to_str(arr).split(' ')):
        idx_data_list.append((i,data))
    return idx_data_list

def create_idx_data(list_:list) -> str:
    """
    입력: 
    [(0, '0.0'),
    (1, '0.19'),
    (2, '0.37')] 

    출력:
    '(0,0.0), (1,0.19), (2,0.37)' 형태
    """
    formatted_items = []
    for item in list_:
        formatted_item = f"({item[0]},{item[1]})"
        formatted_items.append(formatted_item)
    idx_data = ', '.join(formatted_items)
    return idx_data

In [7]:
def create_anomal_series(series_list, gt):
    result_list = []
    result = []
    for diff_anom in gt:
        # ground truth index는 1부터 1000이라 아래와 같이 indexing 해야 out of range error 안 뜸.
        for i in range(diff_anom[0]-1,diff_anom[1]):
            result_list.append(series_list[i])
        result.append(result_list)
        result_list = []
    
    return result # 0.19 0.37 0.54 이런 형태.

In [8]:
def create_anomal_data_prompt(anomal_list):
    result = []
    
    for i, anomaly_group in enumerate(anomal_list, 1):
        result.append(f"anomaly data {i}:")
        
        values = [item[1] for item in anomaly_group]
        result.append(" ".join(values))
        
        if i < len(anomal_list):
            result.append("")
    
    return "\n".join(result)

In [9]:
from prompt import LIMIT_PROMPT,PROMPT


def create_localization_prompt(
        idx,
        series,
        anomaly_locations,
        with_index:bool = False,
        anomaly:bool = False,
):
    gt = create_gt_data(anomaly_locations)
    idx_data = create_idx_data(create_idx_datalist(series[idx]))
    #idx_list = create_idx_datalist(series[idx])
    anomal_text = create_anomal_series(create_idx_datalist(series[idx]), gt[idx]) #j번째 anomal
    
    messages = [
        {
            "role": "user",
            "content": (idx_data if with_index else str(time_series_to_str(series[idx])) ) 
                    + "\n\n" 
                    + LIMIT_PROMPT 
                    + (make_localization_prompt(create_anomal_data_prompt(anomal_text)) if anomaly else PROMPT)
        }
    ]
    if idx == 0:
        print(messages[0]["content"])
    return {
        "messages": messages,
        "temperature": 0.4,
        "stop": ["''''", " – –", "<|endoftext|>", "<|eot_id|>"]
    }


In [10]:
from openai_api import send_openai_request
import json
import time
import os
from loguru import logger
from data.synthetic import SyntheticDataset




def online_AD_with_retries(
    model_name: str,
    data_name: str,
    #request_func: callable,
    variant: str = "standard",
    num_retries: int = 4,
    with_index: bool = False,
    anomaly: bool = False,
):
         
    # Initialize dictionary to store results
    results = {}

    # Configure logger
    log_fn = f"logs/synthetic/{data_name}/{model_name}/" + variant + ".log"
    logger.add(log_fn, format="{time} {level} {message}", level="INFO")
    results_dir = f'{BASE_DIR}/results/synthetic/{data_name}/{model_name}/'
    data_dir = f'{BASE_DIR}/data/synthetic/{data_name}/eval/'
    train_dir = f'{BASE_DIR}/data/synthetic/{data_name}/train/'

    # 실험할 것들 이름 잘 구분되게 수정. variant 뒤쪽에.
    if with_index:
        jsonl_fn = os.path.join(results_dir, variant + 'index-True.jsonl')
    else:
        jsonl_fn = os.path.join(results_dir, variant + '.jsonl')
    os.makedirs(results_dir, exist_ok=True)

    eval_dataset = SyntheticDataset(data_dir)
    eval_dataset.load()

    train_dataset = SyntheticDataset(train_dir)
    train_dataset.load()

    # 함수 안에 있는 변수명과 밖의 변수명은 무조건 구분해서 쓰는게 좋다.
    # text_series ...

    # prepare series and anomaly locations(gt)
    series = []
    anomaly_locations = []
    for idx in range(len(eval_dataset)): #(0부터 399)
        series.append(eval_dataset[idx][1].numpy()) # eval dataset의 문제 series.
        anomaly_locations.append(eval_dataset[idx][0].numpy()) # eval dataset의 gt가 될 것.
    

    if os.path.exists(jsonl_fn):
        logger.error(f"Results file '{jsonl_fn}' already exists. Please delete it first to avoid duplicate entries.")
        print(f"Results file '{jsonl_fn}' already exists. Please delete it and run again.")
        return None

    # Loop over image files

    # 이미 파일 있으면 아예 멈추게. 같은 이름 파일 있으니까 지우고 해라.라고 출력. None return
    # 지우면 안됨.
    # open ('a') -> 파일을 다시 열면 wrtie 가 append되니까..

    # 원래 코드대로 1부터 시작하게끔 수정.
    for i in range(1, len(eval_dataset)+1):
        custom_id = f"{data_name}_{model_name}_{variant}_{str(i).zfill(5)}"
        gt = create_gt_data(anomaly_locations)

        # If there are no anomaly, response is forced to "```json\n[]\n```\n"
        # and others (custom_id, request) are same.
        # 항상 400개 맞춰지게.
        if (anomaly == True) and (len(gt[i-1]) == 0):
            logger.info(f"Setting empty list response for {custom_id} as it has no anomaly locations")
            if with_index:
                request = create_localization_prompt(i-1, series, anomaly_locations, 
                                                        with_index=True, anomaly = True)
            else:
                request = create_localization_prompt(i-1, series, anomaly_locations, 
                                                        with_index=False, anomaly = True)
            response = "```json\n[]\n```\n"
            # Write the result to jsonl
            with open(jsonl_fn, 'a') as f:
                json.dump({'custom_id': custom_id, 'request': request, 'response': response}, f)
                f.write('\n')
            results[custom_id] = response
            continue


        
        # Skip already processed files
        #if custom_id in results:
        #    continue
        
        # Perform anomaly detection with exponential backoff
        for attempt in range(num_retries):
            try:
                if with_index:
                    # index 있고 anomaly 도 주고
                    request = create_localization_prompt(i-1, series, anomaly_locations,
                                                         with_index=True, anomaly=True)
                elif anomaly:
                    # index 없고, anomaly 주고
                    request = create_localization_prompt(i-1, series, anomaly_locations,
                                                         with_index=False, anomaly=True)
                else:
                    #index 없고, anomlay 안 주고 (baseline)
                    request = create_localization_prompt(i-1, series, anomaly_locations,
                                                         with_index=False, anomaly=False)
                response = send_openai_request(request, model_name)
                
                #debugging
                print(response)
                
                
                # Write the result to jsonl
                with open(jsonl_fn, 'a') as f:
                    json.dump({'custom_id': custom_id, 'request': request, 'response': response}, f)
                    f.write('\n')
                # If successful, break the retry loop
                break
            except Exception as e:
                if "503" in str(e):  # Server not up yet, sleep until the server is up again
                    while True:
                        logger.debug("503 error, sleep 30 seconds")
                        time.sleep(30)
                        try:
                            response = send_openai_request(request, model_name)
                            break
                        except Exception as e:
                            if "503" not in str(e):
                                break
                else:
                    logger.error(e)
                    # If an exception occurs, wait and then retry
                    wait_time = 2 ** (attempt + 3)
                    logger.debug(f"Attempt {attempt + 1} failed. Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                    continue
        else:
            logger.error(f"Failed to process {custom_id} after {num_retries} attempts")

In [11]:
for data in [
    #"freq", "point", "range", 
             "trend"]:

    # index data & tell the anomaly
    online_AD_with_retries('gemini-1.5-flash', data, 'localization (indexData)', with_index=True, anomaly=True)
    
    # non-index data & tell the anomaly 
    online_AD_with_retries('gemini-1.5-flash', data, 'localization (non-index)', with_index=False, anomaly=True)
    
    # baseline
    online_AD_with_retries('gemini-1.5-flash', data, 'baseline', with_index=False, anomaly=False)

2025-04-04 13:02:03.952 | INFO     | __main__:online_AD_with_retries:75 - Setting empty list response for trend_gemini-1.5-flash_localization (indexData)_00001 as it has no anomaly locations
2025-04-04 13:02:03.959 | INFO     | __main__:online_AD_with_retries:75 - Setting empty list response for trend_gemini-1.5-flash_localization (indexData)_00002 as it has no anomaly locations
2025-04-04 13:02:03.965 | INFO     | __main__:online_AD_with_retries:75 - Setting empty list response for trend_gemini-1.5-flash_localization (indexData)_00003 as it has no anomaly locations
2025-04-04 13:02:03.976 | INFO     | __main__:online_AD_with_retries:75 - Setting empty list response for trend_gemini-1.5-flash_localization (indexData)_00004 as it has no anomaly locations
2025-04-04 13:02:03.982 | INFO     | __main__:online_AD_with_retries:75 - Setting empty list response for trend_gemini-1.5-flash_localization (indexData)_00005 as it has no anomaly locations
2025-04-04 13:02:03.993 | DEBUG    | gemini_a

Loaded dataset trend with 400 series.
Loaded dataset trend with 400 series.
(0,-0.63), (1,-0.57), (2,-0.52), (3,-0.47), (4,-0.42), (5,-0.38), (6,-0.33), (7,-0.3), (8,-0.26), (9,-0.24), (10,-0.22), (11,-0.2), (12,-0.2), (13,-0.19), (14,-0.2), (15,-0.21), (16,-0.23), (17,-0.25), (18,-0.28), (19,-0.32), (20,-0.36), (21,-0.4), (22,-0.45), (23,-0.5), (24,-0.55), (25,-0.6), (26,-0.65), (27,-0.7), (28,-0.75), (29,-0.79), (30,-0.84), (31,-0.88), (32,-0.91), (33,-0.94), (34,-0.96), (35,-0.98), (36,-0.99), (37,-1.0), (38,-1.0), (39,-0.99), (40,-0.98), (41,-0.96), (42,-0.93), (43,-0.9), (44,-0.86), (45,-0.82), (46,-0.77), (47,-0.72), (48,-0.67), (49,-0.62), (50,-0.57), (51,-0.51), (52,-0.46), (53,-0.41), (54,-0.36), (55,-0.31), (56,-0.27), (57,-0.23), (58,-0.2), (59,-0.18), (60,-0.15), (61,-0.14), (62,-0.13), (63,-0.13), (64,-0.14), (65,-0.15), (66,-0.17), (67,-0.19), (68,-0.22), (69,-0.25), (70,-0.29), (71,-0.34), (72,-0.38), (73,-0.43), (74,-0.48), (75,-0.53), (76,-0.59), (77,-0.64), (78,-0.68)

2025-04-04 13:02:05.424 | ERROR    | __main__:online_AD_with_retries:135 - 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
]
2025-04-04 13:02:05.426 | DEBUG    | __main__:online_AD_with_retries:138 - Attempt 1 failed. Waiting for 8 seconds before retrying...
2025-04-04 13:02:13.446 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************M5D0
2025-04-04 13:02:14.562 | ERROR    | __main__:online_AD_with_retries:135 - 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini A

KeyboardInterrupt: 

# anomaly 가 있을 때 prompt를 한번만 print하게 끔. 

model combination 으로 돌리는 거를 감안.
gemini만 해서 돌리고, metric관련 코드 (evaluation-localization)
jsonl 파일 읽고.. 파싱.. 하는 코드.


- decomposition 실험 돌리기.
gemini api
4월 15일 전까지는 실험 결과도 다 뽑아서 논문 구성 끝남.
metric : accuracy. multi-lable accuracy.
chatgpt 도움.

예측한다음에 anomaly 가 있는지 print하게 끔
항상 확인해야 함.
실수 안하게끔. 최대한. 샘플단위에서 하나씩 확인함.
습관화 해야 한다.
withdraw 하는 것 보다 훨 좋다.

document 


